In [1]:
import tensorflow as tf
import datetime
from Models.Transformers import Transformer, TransformerAbstracter
from Utils.preprocess_scan import generate_data
import tensorboard
%load_ext tensorboard
#%rm -rf ./logs/fit/
tf.config.run_functions_eagerly(False)

/Users/jonathankonig/miniforge3/envs/esbn_abstractor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this notebook, we compare two variants of integrating the esbn into a transformer against the standard transformer. The code for the transformer is adapted from the tensorflow blog.

In [2]:
# Generate data and get vocab sizes
train_ds, val_ds, command_vocab_len, action_vocab_len = generate_data()

2024-04-08 14:16:26.744240: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [3]:
# Define evaluation metrics
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [4]:
# Set number of train / evaluation runs
RUNS = 5    
# Transformer using Relational Abstractor with learned symbol sequence
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

for i in range(RUNS):
    transformer_abstracter = TransformerAbstracter(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        input_vocab_size=command_vocab_len,
        target_vocab_size=action_vocab_len,
        use_esbn=False,
        use_self_attention=True,
        dropout_rate=dropout_rate)

    transformer_abstracter.compile(
        loss=masked_loss,
        optimizer="Adam",
        metrics=[masked_accuracy])

    log_dir = f"logs/fit/transformer_abstracter/run{i}"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    transformer_abstracter.fit(train_ds,
                    epochs=20,
                    validation_data=val_ds,
                    callbacks=[tensorboard_callback])

Epoch 1/20
458/458 [==============================] - 46s 84ms/step - loss: 0.8107 - masked_accuracy: 0.6825 - val_loss: 2.4684 - val_masked_accuracy: 0.4192
Epoch 2/20
458/458 [==============================] - 39s 84ms/step - loss: 0.4606 - masked_accuracy: 0.7957 - val_loss: 2.4734 - val_masked_accuracy: 0.5167
Epoch 3/20
458/458 [==============================] - 39s 85ms/step - loss: 0.3734 - masked_accuracy: 0.8319 - val_loss: 2.3743 - val_masked_accuracy: 0.5640
Epoch 4/20
458/458 [==============================] - 37s 81ms/step - loss: 0.2788 - masked_accuracy: 0.8782 - val_loss: 2.4277 - val_masked_accuracy: 0.5918
Epoch 5/20
458/458 [==============================] - 37s 80ms/step - loss: 0.1608 - masked_accuracy: 0.9350 - val_loss: 2.5577 - val_masked_accuracy: 0.6164
Epoch 6/20
458/458 [==============================] - 37s 82ms/step - loss: 0.1158 - masked_accuracy: 0.9535 - val_loss: 2.4764 - val_masked_accuracy: 0.6509
Epoch 7/20
458/458 [==============================] 

In [5]:
# Transformer using relational abstracter with esbn-generated symbols
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

for i in range(RUNS):
    transformer_abstracter_esbn = TransformerAbstracter(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        input_vocab_size=command_vocab_len,
        target_vocab_size=action_vocab_len,
        use_self_attention=False,
        use_esbn=True,
        dropout_rate=dropout_rate)

    transformer_abstracter_esbn.compile(
        loss=masked_loss,
        optimizer="Adam",
        metrics=[masked_accuracy])

    log_dir = f"logs/fit/transformer_abstracter_esbn/run{i}"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    transformer_abstracter_esbn.fit(train_ds,
                    epochs=20,
                    validation_data=val_ds,
                    callbacks=[tensorboard_callback])

Epoch 1/20
458/458 [==============================] - 46s 82ms/step - loss: 0.8044 - masked_accuracy: 0.6788 - val_loss: 2.8005 - val_masked_accuracy: 0.4108
Epoch 2/20
458/458 [==============================] - 38s 82ms/step - loss: 0.4758 - masked_accuracy: 0.7890 - val_loss: 3.0152 - val_masked_accuracy: 0.4581
Epoch 3/20
458/458 [==============================] - 38s 84ms/step - loss: 0.3580 - masked_accuracy: 0.8430 - val_loss: 3.3077 - val_masked_accuracy: 0.4959
Epoch 4/20
458/458 [==============================] - 37s 82ms/step - loss: 0.2329 - masked_accuracy: 0.9040 - val_loss: 3.4818 - val_masked_accuracy: 0.4992
Epoch 5/20
458/458 [==============================] - 38s 82ms/step - loss: 0.1573 - masked_accuracy: 0.9364 - val_loss: 3.4281 - val_masked_accuracy: 0.5420
Epoch 6/20
458/458 [==============================] - 37s 81ms/step - loss: 0.1208 - masked_accuracy: 0.9519 - val_loss: 3.4158 - val_masked_accuracy: 0.5325
Epoch 7/20
458/458 [==============================] 

In [6]:
# Baseline Transformer model
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

for i in range(RUNS):
    transformer = Transformer(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        input_vocab_size=command_vocab_len,
        target_vocab_size=action_vocab_len,
        dropout_rate=dropout_rate)

    transformer.compile(
        loss=masked_loss,
        optimizer="Adam",
        metrics=[masked_accuracy])

    log_dir = f"logs/fit/transformer/run{i}"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    transformer.fit(train_ds,
                    epochs=20,
                    validation_data=val_ds,
                    callbacks=[tensorboard_callback])


Epoch 1/20
458/458 [==============================] - 22s 42ms/step - loss: 0.8245 - masked_accuracy: 0.6748 - val_loss: 2.7680 - val_masked_accuracy: 0.4486
Epoch 2/20
458/458 [==============================] - 19s 41ms/step - loss: 0.5084 - masked_accuracy: 0.7764 - val_loss: 3.1378 - val_masked_accuracy: 0.4599
Epoch 3/20
458/458 [==============================] - 18s 39ms/step - loss: 0.4132 - masked_accuracy: 0.8151 - val_loss: 2.9219 - val_masked_accuracy: 0.5342
Epoch 4/20
458/458 [==============================] - 18s 40ms/step - loss: 0.3392 - masked_accuracy: 0.8509 - val_loss: 2.9643 - val_masked_accuracy: 0.5828
Epoch 5/20
458/458 [==============================] - 18s 38ms/step - loss: 0.2417 - masked_accuracy: 0.8989 - val_loss: 2.8626 - val_masked_accuracy: 0.6283
Epoch 6/20
458/458 [==============================] - 18s 40ms/step - loss: 0.1774 - masked_accuracy: 0.9266 - val_loss: 2.8539 - val_masked_accuracy: 0.6221
Epoch 7/20
458/458 [==============================] 